---
**License**

 NonLimber

 Mon Jan 25 20:56:00 2020\
 Copyright  2021\
 Sandro Dias Pinto Vitenti <vitenti@uel.br>
 Mariana Penna-Lima <pennalima@unb.br>

---
---

 TestInterp\
 Copyright (C) 2021 Sandro Dias Pinto Vitenti <vitenti@uel.br>
 Copyright (C) 2021 Mariana Penna-Lima <pennalima@unb.br>


 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

In [ ]:
try:
    import gi

    gi.require_version("NumCosmo", "1.0")
    gi.require_version("NumCosmoMath", "1.0")
except:
    pass

import os
import sys
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import scipy.stats
from scipy.interpolate import interp1d, interp2d
import n5k

from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

%matplotlib inline

In [ ]:
__name__ = "NcContext"

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [ ]:
class N5KCalculatorNc(n5k.N5KCalculatorBase):
    name = "NumCosmo"

    def setup(self):
        tpar = self.get_tracer_parameters()
        ker = self.get_tracer_kernels()
        pk = self.get_pk()
        bg = self.get_background()

        # for ki in ker.keys():
        #    print (ki, ker[ki].shape)

        # for pki in pk.keys():
        #    print (pki, pk[pki].shape)

        # for bgi in bg.keys():
        #    print (bgi, bg[bgi].shape)

    def run(self):
        pk = self.get_pk()
        kernels = self.get_tracer_kernels()
        background = self.get_background()
        ells = self.get_ells()

        k = pk["k"]
        pk_z = pk["z"]
        pk_nl = pk["pk_nl"]

        z2chi = interp1d(background["z"], background["chi"])
        chi2z = interp1d(
            background["chi"], background["z"], bounds_error=False, fill_value=1.0e30
        )
        pk_interp = interp2d(
            pk["k"],
            z2chi(pk["z"]),
            pk["pk_nl"],
            kind="cubic",
            bounds_error=False,
            fill_value=0.0,
        )
        self.kernels_interp_cl = [
            interp1d(
                kernels["chi_cl"],
                kernels["kernels_cl"][i],
                fill_value=0.0,
                bounds_error=False,
            )
            for i in range(len(kernels["kernels_cl"]))
        ]
        self.kernels_interp_sh = [
            interp1d(
                kernels["chi_sh"],
                kernels["kernels_sh"][i],
                fill_value=0.0,
                bounds_error=False,
            )
            for i in range(len(kernels["kernels_sh"]))
        ]

        lnk_min = math.log(k[0])
        lnk_max = math.log(k[-1])
        lnk0 = 0.5 * (lnk_max + lnk_min)
        Lk = lnk_max - lnk_min
        lnchi_min = math.log(kernels["chi_cl"][0])
        lnchi_max = math.log(kernels["chi_cl"][-1])
        lnchi0 = 0.5 * (lnchi_max + lnchi_min)
        Lchi = lnchi_max - lnchi_min
        lnPchi_max = math.log(z2chi(pk_z[-1]))
        L = max(Lk, Lchi)

        # print ("Pk:  lnk_min:   % 22.15g lnk_max:   % 22.15g, Lk   = % 22.15g" % (lnk_min, lnk_max, lnk_max - lnk_min))
        # print ("Pk:  lnchi_max: % 22.15g" % (lnPchi_max))
        # print ("Kcl: lnchi_min: % 22.15g lnchi_max: % 22.15g, Lchi = % 22.15g" % (lnchi_min, lnchi_max, lnchi_max - lnchi_min))

        NT = 8

        self.lnk_a = lnk0 + np.arange(-NT / 2, NT / 2) * L / NT
        self.lnchi_a = lnchi0 + np.arange(-NT / 2, NT / 2) * L / NT
        self.pk_a = pk_interp(np.exp(self.lnk_a), chi2z(np.exp(self.lnchi_a))).T
        self.NT = NT
        self.lnk0 = lnk0
        self.lnchi0 = lnchi0
        self.L = L
        # print (self.kernels_interp_cl[0] (np.exp (self.lnchi_a)))
        # print (self.pk_a[:,None] * self.kernels_interp_cl[0] (np.exp (self.lnchi_a)))

        # print (lnk_a)
        # print (lnchi_a)
        # print (FM)

        tp = np.exp(
            -2.0
            * np.pi
            * 1j
            * (self.lnk0 + self.lnchi0)
            / cal_nl.L
            * np.arange(cal_nl.NT / 2)
        )
        tn = np.exp(
            -2.0
            * np.pi
            * 1j
            * (self.lnk0 + self.lnchi0)
            / cal_nl.L
            * np.arange(-cal_nl.NT / 2, 0)
        )

        self.center = np.concatenate((tp, tn))

        nl = len(self.get_ells())
        ngg, ngs, nss = self.get_num_cls()
        self.cls_gg = np.zeros((ngg, nl))
        self.cls_gs = np.zeros((ngs, nl))
        self.cls_ss = np.zeros((nss, nl))

In [ ]:
os.chdir("/home/sandro/Projects/N5K")

cal_nl = N5KCalculatorNc("tests/config_nl.yml")
cal_nl.setup()
cal_nl.run()
cal_nl.write_output()
cal_nl.teardown()

x, y = np.meshgrid(cal_nl.lnk_a, cal_nl.lnchi_a)

fig = plt.figure(figsize=(18, 18))
ax = fig.add_subplot(111)

W = cal_nl.kernels_interp_cl[0](np.exp(cal_nl.lnchi_a))
P = np.sqrt(cal_nl.pk_a)
PW = P * W

p = ax.imshow(
    PW,
    interpolation="quadric",
    origin="lower",
    extent=(cal_nl.lnk_a[0], cal_nl.lnk_a[-1], cal_nl.lnchi_a[0], cal_nl.lnchi_a[-1]),
    norm=matplotlib.colors.SymLogNorm(10 ** (-1), base=10.0),
)
plt.colorbar(p)
pass

In [ ]:
TPW = np.fft.fft2(PW)
TPWs = np.fft.fftshift(TPW)

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111)

p = ax.imshow(np.real(TPWs), norm=matplotlib.colors.SymLogNorm(10 ** (-1), base=10.0))
plt.colorbar(p)

TPW = TPWs

pass

In [ ]:
index_a = np.concatenate((np.arange(0, cal_nl.NT / 2), np.arange(-cal_nl.NT / 2, 0)))
index_a = np.concatenate((np.arange(-cal_nl.NT / 2, 0), np.arange(0, cal_nl.NT / 2)))

for m in np.arange(cal_nl.NT):
    for n in np.arange(cal_nl.NT):
        v1 = np.exp(+2.0 * np.pi * m * 1j / cal_nl.NT * index_a)
        v2 = np.exp(+2.0 * np.pi * n * 1j / cal_nl.NT * index_a)

        test = (TPW.dot(v2)).dot(v1) / cal_nl.NT**2

        if PW[m, n] == 0.0:
            continue

        reltol = np.real(test / PW[m, n] - 1.0)

        print(
            "%3d %3d % 22.15g % 22.15g % 8.2e % 8.2e"
            % (m, n, np.real(test), PW[m, n], np.imag(test), reltol)
        )

In [ ]:
fft_jl = Ncm.FftlogSBesselJ.new(15, cal_nl.lnchi0, cal_nl.lnk0, cal_nl.L, cal_nl.NT)
fft_jl.set_padding(0.0)

In [ ]:
fft_jl.get_full_size()

In [ ]:
TPWc = TPW * cal_nl.center

fft_jl.set_q(0.5)

ep = np.array(fft_jl.get_Ym())
ep = ep[0::2] + ep[1::2] * 1j


print(fft_jl.get_q())

print(ep.dot(np.conj(ep)))
len(ep)
TPW.shape

In [ ]:
MF = np.zeros_like(TPWc)
for n in range(cal_nl.NT):
    for r in range(cal_nl.NT):
        li = min(n + 1, cal_nl.NT - r)
        MF[n, r] = np.sum([TPWc[n - l, n] * TPWc[r + l, r] for l in range(li)])

In [ ]:
MF2 = np.zeros_like(TPWc)
for n in range(cal_nl.NT):
    for r in range(cal_nl.NT):
        MF2[n, r] = 0.0
        for m in range(cal_nl.NT):
            for q in range(cal_nl.NT):
                for s in range(cal_nl.NT):
                    if (n == m + s) and (r == q - s):
                        MF2[n, r] = MF2[n, r] + TPWc[m, n] * TPWc[q, r]

In [ ]:
for n in range(cal_nl.NT):
    for r in range(cal_nl.NT):
        if MF[n, r] != MF2[n, r]:
            print(
                "%4d %4d % 22.15g % 22.15gi"
                % (n, r, np.real(MF[n, r] - MF2[n, r]), np.imag(MF[n, r] - MF2[n, r]))
            )

In [ ]:
MF.dot(ep).dot(ep)

In [ ]:
cal_nl.L

In [ ]:
ep[17]

In [ ]:
for i in range(-10, 0):
    print(i)

In [ ]:
np.concatenate(([1, 3], [2]))

In [ ]:
np.concatenate((np.arange(-cal_nl.NT / 2, 0), np.arange(cal_nl.NT / 2)))